In [31]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import matplotlib
from skimage.transform import resize
from imageio import imread, imwrite
import pandas as pd
import glob
import pickle
import numpy as np
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense

from lib.utils import load_gtsrb

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(sess)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train[:, :, :, np.newaxis] / 255.
X_test = X_test[:, :, :, np.newaxis] / 255.
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1)

11493376/11490434 [==============================] - 2s 0us/step


In [3]:
from image_sim_models import SiameseNetwork

name = "sn_v1"
sn = SiameseNetwork(name, [28, 28, 1], 100, 
                    learning_rate=1e-4, reg=0, 
                    load_model=True, 
                    save_path="model/" + name + ".h5")
# data = (X_train, y_train, X_val, y_val)
# sn.train_model(sess, data, dataaug=False, n_epoch=10, batch_size=128)

In [4]:
sn.eval_model(sess, (X_test, y_test), batch_size=128)

(array([0.57563984, 0.4750827 , 0.59330618, ..., 0.4234283 , 0.63484615,
        0.54132634]),
 0.5154948548607107,
 0.5799736886083147,
 0.2014518481731415)

In [17]:
X_test_embed = sn.get_embed(sess, X_test)
X_train_embed = sn.get_embed(sess, X_train)
X_val_embed = sn.get_embed(sess, X_val)

In [6]:
i = 5
dist = np.sum((X_test_embed[i] - X_test_embed)**2, -1)
dist_same = dist[np.where(y_test == y_test[i])[0]] 
dist_diff = dist[np.where(y_test != y_test[i])[0]] 

In [12]:
thres = 0.25
print('Average distance from samples of same class: ', np.mean(dist_same))
print('Average distance from samples of diff class: ', np.mean(dist_diff))
print(np.mean(dist_same < thres))
print(np.mean(dist_diff >= thres))

Average distance from samples of same class:  0.1901354734687922
Average distance from samples of diff class:  0.3121024205663281
0.6925110132158591
0.9010716300056402


Do kNN with different k's, calculate accuracy on test set

In [13]:
k = 5

n_correct = 0
for i, x in enumerate(X_test_embed):
    dist = np.sum((x - X_train_embed)**2, -1)
    ind = np.argsort(dist)
    y = np.argmax(np.bincount(y_train[ind][:k].flatten()))
    if y == y_test[i]:
        n_correct += 1
print(n_correct / len(y_test))

0.9566


In [14]:
k = 10

n_correct = 0
for i, x in enumerate(X_test_embed):
    dist = np.sum((x - X_train_embed)**2, -1)
    ind = np.argsort(dist)
    y = np.argmax(np.bincount(y_train[ind][:k].flatten()))
    if y == y_test[i]:
        n_correct += 1
print(n_correct / len(y_test))

0.9553


Find `size` nearest neighbors to each samples in order

In [ ]:
size = 200

nn_test = np.zeros((X_test.shape[0], size))
for i, x in enumerate(X_test_embed):
    dist = np.sum((x - X_train_embed)**2, -1)
    ind = np.argsort(dist)
    nn_test[i] = y_train[ind][:size].flatten()
    
nn_val = np.zeros((X_val.shape[0], size))
for i, x in enumerate(X_val_embed):
    dist = np.sum((x - X_train_embed)**2, -1)
    ind = np.argsort(dist)
    nn_val[i] = y_train[ind][:size].flatten()
    
# For NN of train set, we exclude itself
nn_train = np.zeros((X_train.shape[0], size))
for i, x in enumerate(X_train_embed):
    dist = np.sum((x - X_train_embed)**2, -1)
    ind = np.argsort(dist)
    nn_train[i] = y_train[ind][1:size+1].flatten()

In [23]:
pickle.dump([nn_train, nn_test, nn_val], open('nn_{}.pkl'.format(size), 'wb'))

In [29]:
# Accuracy using 200-NN
n_correct = 0
for i, n in enumerate(nn_test):
    y = np.argmax(np.bincount(n.astype(np.int32)))
    if y == y_test[i]:
        n_correct += 1
print(n_correct / len(y_test))

0.9212


Try training a network to assign weights to each of the neighbors and outputs weighted average (which is not quite the correct way).

In [40]:
model = Sequential()
model.add(Dense(1, activation=None))
model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])
model.fit(nn_train, y_train,
          batch_size=128,
          epochs=30,
          verbose=1,
          validation_data=(nn_test, y_test))

Train on 54000 samples, validate on 10000 samples
Epoch 1/30
54000/54000 [==============================] - 1s 26us/step - loss: 6.1691 - acc: 0.4283 - val_loss: 3.7771 - val_acc: 0.4801
Epoch 2/30
54000/54000 [==============================] - 1s 22us/step - loss: 3.3171 - acc: 0.4853 - val_loss: 2.8768 - val_acc: 0.5014
Epoch 3/30
54000/54000 [==============================] - 1s 23us/step - loss: 2.3718 - acc: 0.5270 - val_loss: 2.0259 - val_acc: 0.5686
Epoch 4/30
54000/54000 [==============================] - 1s 23us/step - loss: 1.6338 - acc: 0.5841 - val_loss: 1.4304 - val_acc: 0.6333
Epoch 5/30
54000/54000 [==============================] - 1s 23us/step - loss: 1.1064 - acc: 0.6626 - val_loss: 1.0434 - val_acc: 0.6993
Epoch 6/30
54000/54000 [==============================] - 1s 23us/step - loss: 0.7881 - acc: 0.7416 - val_loss: 0.8101 - val_acc: 0.7827
Epoch 7/30
54000/54000 [==============================] - 1s 23us/step - loss: 0.6187 - acc: 0.8127 - val_loss: 0.6982 - val_acc

In [41]:
model.layers[0].get_weights()

[array([[ 3.92268747e-01],
        [ 1.69600517e-01],
        [ 1.18026845e-01],
        [ 1.16576776e-01],
        [ 5.30240387e-02],
        [ 6.43926486e-02],
        [ 3.69208492e-02],
        [ 3.52639779e-02],
        [ 1.10596260e-02],
        [ 1.86302550e-02],
        [ 1.89090166e-02],
        [ 1.27966767e-02],
        [ 2.29013022e-02],
        [ 9.50070377e-03],
        [ 1.75989754e-02],
        [ 1.16670411e-02],
        [-8.45692266e-05],
        [ 5.15874662e-03],
        [ 6.45573810e-03],
        [ 2.87777744e-03],
        [ 1.50318397e-02],
        [ 6.19285181e-03],
        [ 9.49761178e-03],
        [ 4.25699260e-03],
        [ 2.76928628e-03],
        [-5.90716721e-03],
        [ 5.89188328e-03],
        [ 4.71845455e-03],
        [ 6.78645913e-03],
        [ 1.74808130e-03],
        [ 9.55132116e-03],
        [-1.56316552e-02],
        [-5.60824107e-03],
        [-1.03784231e-02],
        [-4.83850343e-03],
        [-3.49694095e-03],
        [ 3.29340412e-03],
 

Train a network to try to recognize the digit based on patterns of its neighbors (which is also not quite correct).

In [42]:
model = Sequential()
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])
model.fit(nn_train, y_train,
          batch_size=128,
          epochs=30,
          verbose=1,
          validation_data=(nn_test, y_test))

Train on 54000 samples, validate on 10000 samples
Epoch 1/30
54000/54000 [==============================] - 2s 41us/step - loss: 1.6449 - acc: 0.4899 - val_loss: 1.0364 - val_acc: 0.6759
Epoch 2/30
54000/54000 [==============================] - 2s 35us/step - loss: 0.9183 - acc: 0.6917 - val_loss: 0.8889 - val_acc: 0.7611
Epoch 3/30
54000/54000 [==============================] - 2s 35us/step - loss: 0.7736 - acc: 0.7518 - val_loss: 0.8052 - val_acc: 0.7621
Epoch 4/30
54000/54000 [==============================] - 2s 35us/step - loss: 0.6897 - acc: 0.7829 - val_loss: 0.7404 - val_acc: 0.7969
Epoch 5/30
54000/54000 [==============================] - 2s 35us/step - loss: 0.6303 - acc: 0.8076 - val_loss: 0.6459 - val_acc: 0.8358
Epoch 6/30
54000/54000 [==============================] - 2s 35us/step - loss: 0.5547 - acc: 0.8394 - val_loss: 0.6337 - val_acc: 0.8288
Epoch 7/30
54000/54000 [==============================] - 2s 35us/step - loss: 0.5233 - acc: 0.8504 - val_loss: 0.6256 - val_acc

Train network directly on embedding

In [43]:
model = Sequential()
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])
model.fit(X_train_embed, y_train,
          batch_size=128,
          epochs=30,
          verbose=1,
          validation_data=(X_test_embed, y_test))

Train on 54000 samples, validate on 10000 samples
Epoch 1/30
54000/54000 [==============================] - 2s 34us/step - loss: 0.7464 - acc: 0.8220 - val_loss: 0.3449 - val_acc: 0.8969
Epoch 2/30
54000/54000 [==============================] - 2s 30us/step - loss: 0.3279 - acc: 0.8988 - val_loss: 0.2837 - val_acc: 0.9132
Epoch 3/30
54000/54000 [==============================] - 2s 30us/step - loss: 0.2914 - acc: 0.9100 - val_loss: 0.2666 - val_acc: 0.9178
Epoch 4/30
54000/54000 [==============================] - 2s 30us/step - loss: 0.2695 - acc: 0.9166 - val_loss: 0.2476 - val_acc: 0.9242
Epoch 5/30
54000/54000 [==============================] - 2s 30us/step - loss: 0.2519 - acc: 0.9211 - val_loss: 0.2326 - val_acc: 0.9286
Epoch 6/30
54000/54000 [==============================] - 2s 29us/step - loss: 0.2357 - acc: 0.9276 - val_loss: 0.2207 - val_acc: 0.9333
Epoch 7/30
54000/54000 [==============================] - 2s 30us/step - loss: 0.2204 - acc: 0.9322 - val_loss: 0.2108 - val_acc